In [14]:
import pandas as pd 
import numpy as np
import requests

In [ ]:
def json_to_df(response):
    """
    convert response to dataframe
    """
    
    return pd.DataFrame(response.json()[1:], columns = response.json()[0])


def df_to_csv(df, filename):
    """
    convert df to csv file
    """
    return df.to_csv(filename, index = False)



census_api_key = "2bdd0ae628fa3f0bacc71a11c1af43240f11a8f1"

# Population Data API links for 2019-2015


https://api.census.gov/data/2019/pep/population/examples.html

base API I started with
https://api.census.gov/data/2019/pep/charagegroups?get=NAME,POP&for=county:*&in=state:*&key=YOUR_KEY_GOES_HERE

overview of variables
https://api.census.gov/data/2019/pep/charagegroups/variables.html

description of values in each column
https://www.census.gov/data/developers/data-sets/popest-popproj/popest/popest-vars.html

examples of api calls 
https://api.census.gov/data/2019/pep/charagegroups/examples.html

## Population by race 2019-2014 NC per county

In [245]:
url = f"https://api.census.gov/data/2019/pep/charagegroups?get=NAME,POP&RACE=1,2,3,4,5,6&HISP=1,2&AGEGROUP=1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18&DATE_CODE=7,8,9,10,11,12&DATE_DESC&for=county:*&in=state:37&key={census_api_key}"
response = requests.request("GET", url)

In [246]:
df_2019_to_2014 = json_to_df(response)

In [247]:
df_2019_to_2014

,NAME,POP,RACE,HISP,AGEGROUP,DATE_CODE,DATE_DESC,state,county
0,"Greene County, North Carolina",429,1,1,1,7,7/1/2014 population estimate,37,079
1,"Greene County, North Carolina",240,1,2,1,7,7/1/2014 population estimate,37,079
2,"Greene County, North Carolina",349,2,1,1,7,7/1/2014 population estimate,37,079
3,"Greene County, North Carolina",48,2,2,1,7,7/1/2014 population estimate,37,079
4,"Greene County, North Carolina",2,3,1,1,7,7/1/2014 population estimate,37,079
...,...,...,...,...,...,...,...,...,...
129595,"Alamance County, North Carolina",0,4,2,18,12,7/1/2019 population estimate,37,001
129596,"Alamance County, North Carolina",0,5,1,18,12,7/1/2019 population estimate,37,001
129597,"Alamance County, North Carolina",0,5,2,18,12,7/1/2019 population estimate,37,001
129598,"Alamance County, North Carolina",19,6,1,18,12,7/1/2019 population estimate,37,001


# Data Cleaning

### drop columns 

In [248]:
df_2019_to_2014.drop(columns=['DATE_CODE'], inplace = True)

In [249]:
df_2019_to_2014.head()

,NAME,POP,RACE,HISP,AGEGROUP,DATE_DESC,state,county
0,"Greene County, North Carolina",429,1,1,1,7/1/2014 population estimate,37,079
1,"Greene County, North Carolina",240,1,2,1,7/1/2014 population estimate,37,079
2,"Greene County, North Carolina",349,2,1,1,7/1/2014 population estimate,37,079
3,"Greene County, North Carolina",48,2,2,1,7/1/2014 population estimate,37,079
4,"Greene County, North Carolina",2,3,1,1,7/1/2014 population estimate,37,079


### rename the columns

In [250]:
df_2019_to_2014.rename(columns = {"NAME":"county_name", "POP":"population", "RACE":"race", "HISP":"hispanic_or_not", "AGEGROUP":"age_group", "DATE_DESC":"year", "county":"county_code"}, inplace = True)

In [251]:
df_2019_to_2014

,county_name,population,race,hispanic_or_not,age_group,year,state,county_code
0,"Greene County, North Carolina",429,1,1,1,7/1/2014 population estimate,37,079
1,"Greene County, North Carolina",240,1,2,1,7/1/2014 population estimate,37,079
2,"Greene County, North Carolina",349,2,1,1,7/1/2014 population estimate,37,079
3,"Greene County, North Carolina",48,2,2,1,7/1/2014 population estimate,37,079
4,"Greene County, North Carolina",2,3,1,1,7/1/2014 population estimate,37,079
...,...,...,...,...,...,...,...,...
129595,"Alamance County, North Carolina",0,4,2,18,7/1/2019 population estimate,37,001
129596,"Alamance County, North Carolina",0,5,1,18,7/1/2019 population estimate,37,001
129597,"Alamance County, North Carolina",0,5,2,18,7/1/2019 population estimate,37,001
129598,"Alamance County, North Carolina",19,6,1,18,7/1/2019 population estimate,37,001


### we need to update the values (i.e. county_name), categorize the data to proper descriptions (i.e. race), and change the data types (i.e. hispanic_or_not)

In [252]:
df_2019_to_2014.dtypes

county_name        object
population         object
race               object
hispanic_or_not    object
age_group          object
year               object
state              object
county_code        object
dtype: object

### first strip values to a more concise format

In [253]:
def remove_extra_description(string):
    if 'County' in string:
        return string[:string.find(' Co')]
    elif 'population' in string:
        return string[:string.find(' p')]
    else:
        pass

In [254]:
df_2019_to_2014.loc[:, 'county_name'] = df_2019_to_2014['county_name'].apply(remove_extra_description)
df_2019_to_2014.loc[:, 'year'] = df_2019_to_2014['year'].apply(remove_extra_description)

In [255]:
df_2019_to_2014[df_2019_to_2014['age_group'] == '18']

,county_name,population,race,hispanic_or_not,age_group,year,state,county_code
204,Greene,256,1,1,18,7/1/2014,37,079
205,Greene,6,1,2,18,7/1/2014,37,079
206,Greene,128,2,1,18,7/1/2014,37,079
207,Greene,1,2,2,18,7/1/2014,37,079
208,Greene,2,3,1,18,7/1/2014,37,079
...,...,...,...,...,...,...,...,...
129595,Alamance,0,4,2,18,7/1/2019,37,001
129596,Alamance,0,5,1,18,7/1/2019,37,001
129597,Alamance,0,5,2,18,7/1/2019,37,001
129598,Alamance,19,6,1,18,7/1/2019,37,001


In [256]:
df_2019_to_2014['race'].unique().tolist()

['1', '2', '3', '4', '5', '6']

### update the values in race, hispanic_or_not, age_group, and state  

In [257]:
def update_race_description(string):
    
    reference_table = {
        '1':'white',
        '2':'black',
        '3':'american indian and alaska native',
        '4':'asian',
        '5':'native hawaiian and other pacific islander',
        '6':'two or more race'
    }
    
    if string in reference_table:
        return reference_table[string]
    

    
def update_hispanic_or_not(string):
    
    if '1' in string:
        return 'False'
    elif '2' in string:
        return 'True'
    

def update_age_group(string):
    
    reference_table = {
        '1': '0 to 4',
        '2': '5 to 9',
        '3': '10 to 14',
        '4': '15 to 19',
        '5': '20 to 24',
        '6': '25 to 29', 
        '7': '30 to 34',
        '8': '35 to 39',
        '9': '40 to 44',
        '10': '45 to 49',
        '11': '50 to 54',
        '12': '55 to 59',
        '13': '60 to 64',
        '14': '65 to 69',
        '15': '70 to 74',
        '16': '75 to 79',
        '17': '80 to 84',
        '18': '85 and up'
    }
    
    if string in reference_table:
        return reference_table[string]
    

def update_state(string):
    
    if '37' in string:
        return 'NC'
    else:
        pass

def binning_age_groups(string):
    
    if string in ['0 to 4','5 to 9','10 to 14','15 to 19']:
        return 'minor'
    elif string in ['20 to 24','25 to 29','30 to 34','35 to 39','40 to 44','45 to 49','50 to 54','55 to 59','60 to 64',]:
        return 'adult'
    elif string in ['65 to 69','70 to 74','75 to 79','80 to 84','85 and up']:
        return 'elder'
    else:
        pass


In [258]:
update_race_description('3')

'american indian and alaska native'

In [259]:
df_2019_to_2014.loc[:, 'race'] = df_2019_to_2014['race'].apply(update_race_description)
df_2019_to_2014.loc[:, 'hispanic_or_not'] = df_2019_to_2014['hispanic_or_not'].apply(update_hispanic_or_not)
df_2019_to_2014.loc[:, 'age_group'] = df_2019_to_2014['age_group'].apply(update_age_group)
df_2019_to_2014.loc[:, 'state'] = df_2019_to_2014['state'].apply(update_state)

In [260]:
df_2019_to_2014

,county_name,population,race,hispanic_or_not,age_group,year,state,county_code
0,Greene,429,white,False,0 to 4,7/1/2014,NC,079
1,Greene,240,white,True,0 to 4,7/1/2014,NC,079
2,Greene,349,black,False,0 to 4,7/1/2014,NC,079
3,Greene,48,black,True,0 to 4,7/1/2014,NC,079
4,Greene,2,american indian and alaska native,False,0 to 4,7/1/2014,NC,079
...,...,...,...,...,...,...,...,...
129595,Alamance,0,asian,True,85 and up,7/1/2019,NC,001
129596,Alamance,0,native hawaiian and other pacific islander,False,85 and up,7/1/2019,NC,001
129597,Alamance,0,native hawaiian and other pacific islander,True,85 and up,7/1/2019,NC,001
129598,Alamance,19,two or more race,False,85 and up,7/1/2019,NC,001


### create additional column that bins age groups 

In [261]:
df_2019_to_2014['age_status'] = df_2019_to_2014['age_group'].apply(binning_age_groups)

In [262]:
df_2019_to_2014

,county_name,population,race,hispanic_or_not,age_group,year,state,county_code,age_status
0,Greene,429,white,False,0 to 4,7/1/2014,NC,079,minor
1,Greene,240,white,True,0 to 4,7/1/2014,NC,079,minor
2,Greene,349,black,False,0 to 4,7/1/2014,NC,079,minor
3,Greene,48,black,True,0 to 4,7/1/2014,NC,079,minor
4,Greene,2,american indian and alaska native,False,0 to 4,7/1/2014,NC,079,minor
...,...,...,...,...,...,...,...,...,...
129595,Alamance,0,asian,True,85 and up,7/1/2019,NC,001,elder
129596,Alamance,0,native hawaiian and other pacific islander,False,85 and up,7/1/2019,NC,001,elder
129597,Alamance,0,native hawaiian and other pacific islander,True,85 and up,7/1/2019,NC,001,elder
129598,Alamance,19,two or more race,False,85 and up,7/1/2019,NC,001,elder


### change data types of population, hispanic_or_not, and year

In [263]:
df_2019_to_2014.dtypes

county_name        object
population         object
race               object
hispanic_or_not    object
age_group          object
year               object
state              object
county_code        object
age_status         object
dtype: object

In [264]:
df_2019_to_2014['population'] = df_2019_to_2014['population'].astype('int64')
df_2019_to_2014['hispanic_or_not'] = df_2019_to_2014['hispanic_or_not'].map({'True':True, 'False':False})
pd.to_datetime(df_2019_to_2014.year, format = '%m/%d/%Y')
df_2019_to_2014['year'] = pd.DatetimeIndex(df_2019_to_2014['year']).year

In [265]:
df_2019_to_2014

,county_name,population,race,hispanic_or_not,age_group,year,state,county_code,age_status
0,Greene,429,white,False,0 to 4,2014,NC,079,minor
1,Greene,240,white,True,0 to 4,2014,NC,079,minor
2,Greene,349,black,False,0 to 4,2014,NC,079,minor
3,Greene,48,black,True,0 to 4,2014,NC,079,minor
4,Greene,2,american indian and alaska native,False,0 to 4,2014,NC,079,minor
...,...,...,...,...,...,...,...,...,...
129595,Alamance,0,asian,True,85 and up,2019,NC,001,elder
129596,Alamance,0,native hawaiian and other pacific islander,False,85 and up,2019,NC,001,elder
129597,Alamance,0,native hawaiian and other pacific islander,True,85 and up,2019,NC,001,elder
129598,Alamance,19,two or more race,False,85 and up,2019,NC,001,elder


In [269]:
df_2019_to_2014.dtypes

county_name        object
population          int64
race               object
hispanic_or_not      bool
age_group          object
year                int64
state              object
county_code        object
age_status         object
dtype: object

### time to check for Null values 

In [278]:
# gives me true or false for any null values 
df_2019_to_2014.isnull().values.any()

False

In [280]:
# counts nulls in each column
df_2019_to_2014.isnull().sum()

county_name        0
population         0
race               0
hispanic_or_not    0
age_group          0
year               0
state              0
county_code        0
age_status         0
dtype: int64

In [281]:
# print distinct values for each column in case null is identified differently
for col in df_2019_to_2014:
    print(df_2019_to_2014[col].unique())


['Greene' 'Haywood' 'Catawba' 'Mitchell' 'Chowan' 'Alleghany' 'Caldwell'
 'Cleveland' 'Edgecombe' 'Craven' 'Granville' 'Harnett' 'Hoke' 'Henderson'
 'Hyde' 'Northampton' 'Pitt' 'Iredell' 'Mecklenburg' 'Cherokee' 'Rowan'
 'Vance' 'Union' 'Forsyth' 'Currituck' 'Wilkes' 'Dare' 'Nash' 'Alexander'
 'Wilson' 'Bladen' 'Carteret' 'Warren' 'Bertie' 'Pamlico' 'Madison'
 'Gates' 'Graham' 'Martin' 'Polk' 'Perquimans' 'Randolph' 'Clay' 'Stokes'
 'Rockingham' 'Washington' 'Camden' 'Cumberland' 'Guilford' 'Jackson'
 'Gaston' 'Pasquotank' 'Moore' 'Wake' 'Robeson' 'Stanly' 'Chatham' 'Swain'
 'Franklin' 'Yancey' 'Anson' 'Transylvania' 'McDowell' 'Macon' 'Johnston'
 'Jones' 'Person' 'Brunswick' 'Ashe' 'Columbus' 'Beaufort' 'Onslow'
 'Lenoir' 'Lincoln' 'Montgomery' 'Orange' 'Richmond' 'Rutherford' 'Surry'
 'Yadkin' 'Sampson' 'Avery' 'Buncombe' 'Caswell' 'Burke' 'Duplin'
 'Cabarrus' 'Davidson' 'Davie' 'Durham' 'Hertford' 'Lee' 'New Hanover'
 'Halifax' 'Pender' 'Wayne' 'Scotland' 'Watauga' 'Tyrrell' 'Alaman

In [287]:
df_to_csv(df_2019_to_2014, 'df_2019_to_2014.csv')

# Load Data to Postgresql DB 

In [1]:
import psycopg2
from configparser import ConfigParser

### connect to database

In [2]:
def config(filename='database.ini', section='postgresql'):
     # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

def connect(arg):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)

        # create a cursor
        cur = conn.cursor()
        
    # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
    # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')


In [3]:
connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 14.7 on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit',)
Database connection closed.
